In [1]:
def getSource(scalar, precision, suffix=''):
    R = RealField(precision)
    P.<u> = R[]
    def getLegendre(n):
        return P(legendre_P(n, u))

    def strScalar(v):
        return str(v)+suffix
    s = "namespace matslise::legendre {\n"
    for n in [2, 4, .. 18]:
        r = getLegendre(n)
        rd = r.diff()
        xs = [v for v, m in r.roots()]
        ws = [2/(1-x^2)/rd(x)^2 for x in xs]

        rows = []
        for i in range(n):
            ls = map(getLegendre(i), xs)
            rows.extend([w*f*R((i+1/2)/2^i) for w, f in zip(ws, ls)])
        tn = 'Eigen::Array<' + scalar + ', %d, 1>'%n
        tw = 'Eigen::Array<' + scalar + ', %d, %d>'%(n, n)
        s += '    template<> inline const ' + tn + ' nodes<' + scalar + ', ' + str(n) + '> = (' + tn + '()'
        s += ' << ' + ', '.join(map(strScalar, xs))+').finished();\n'
        s += '    template<> inline const ' + tw + ' weights<' + scalar + ', ' + str(n) + '> = (' + tw + '()'
        s += ' << ' + ', '.join(map(strScalar, rows))+').finished();\n'
    s += "}\n"
    return s

In [2]:
s = ''
s += getSource('double', 64)
s += '\n'
s += '#ifdef MATSLISE_long_double\n'
s += getSource('long double', 80, 'l')
s += '#endif\n'
s += '\n'
s += '#ifdef MATSLISE_float128\n'
s += '#include <boost/multiprecision/float128.hpp>\n\n'
s += getSource('boost::multiprecision::float128', 128, 'q')
s += '#endif'
s += '\n'
print s

namespace matslise::legendre {
    template<> inline const Eigen::Array<double, 2, 1> nodes<double, 2> = (Eigen::Array<double, 2, 1>() << -0.577350269189625765, 0.577350269189625765).finished();
    template<> inline const Eigen::Array<double, 2, 2> weights<double, 2> = (Eigen::Array<double, 2, 2>() << 0.500000000000000000, 0.500000000000000000, -0.433012701892219323, 0.433012701892219323).finished();
    template<> inline const Eigen::Array<double, 4, 1> nodes<double, 4> = (Eigen::Array<double, 4, 1>() << -0.861136311594052575, -0.339981043584856265, 0.339981043584856265, 0.861136311594052575).finished();
    template<> inline const Eigen::Array<double, 4, 4> weights<double, 4> = (Eigen::Array<double, 4, 4>() << 0.173927422568726929, 0.326072577431273071, 0.326072577431273071, 0.173927422568726929, -0.224662828733840527, -0.166287742739232104, 0.166287742739232104, 0.224662828733840527, 0.133127010504727875, -0.133127010504727875, -0.133127010504727875, 0.133127010504727875, -0.046378

In [3]:
with open('legendre_data.h','w') as f:
    f.write(s)